In [7]:
%%writefile bu.cu
#include <iostream>
#include <vector>
#include <cstdlib>  // For rand()
#include <ctime>    // For clock()

// Sequential Bubble Sort
void bubbleSortSequential(int* arr, int size) {
    for (int i = 0; i < size - 1; i++) {
        for (int j = 0; j < size - i - 1; j++) {
            if (arr[j] > arr[j + 1]) {
                int temp = arr[j];
                arr[j] = arr[j + 1];
                arr[j + 1] = temp;
            }
        }
    }
}

// Parallel Bubble Sort using CUDA
__global__ void bubbleSortParallel(int* arr, int size) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < size) {
        for (int i = 0; i < size - 1; i++) {
            for (int j = 0; j < size - i - 1; j++) {
                if (arr[j] > arr[j + 1]) {
                    int temp = arr[j];
                    arr[j] = arr[j + 1];
                    arr[j + 1] = temp;
                }
            }
        }
    }
}

int main() {
    const int size = 1000;
    int arr[size];
    for (int i = 0; i < size; i++) {
        arr[i] = rand() % 1000;  // Random numbers between 0 and 999
    }

    // Measure sequential bubble sort time
    clock_t startSeqBubble = clock();
    bubbleSortSequential(arr, size);
    clock_t endSeqBubble = clock();
    double timeSeqBubble = double(endSeqBubble - startSeqBubble) / CLOCKS_PER_SEC * 1000;

    // Measure parallel bubble sort time
    int* d_arr;
    cudaMalloc(&d_arr, size * sizeof(int));
    cudaMemcpy(d_arr, arr, size * sizeof(int), cudaMemcpyHostToDevice);

    clock_t startParBubble = clock();
    bubbleSortParallel<<<(size + 255) / 256, 256>>>(d_arr, size);
    cudaDeviceSynchronize();
    clock_t endParBubble = clock();
    double timeParBubble = double(endParBubble - startParBubble) / CLOCKS_PER_SEC * 1000;

    // Print execution times
    std::cout << "Parallel Bubble Sort Time: " << timeSeqBubble << " milliseconds" << std::endl;
    std::cout << "Sequential Bubble Sort Time: " << timeParBubble << " milliseconds" << std::endl;

    // Free device memory
    cudaFree(d_arr);

    return 0;
}


Overwriting bu.cu


In [8]:
!nvcc bu.cu -o bu
!./bu

Parallel Bubble Sort Time: 2.318 milliseconds
Sequential Bubble Sort Time: 41.35 milliseconds


In [9]:
%%writefile mer.cu
#include <iostream>
#include <vector>
#include <cstdlib>  // For rand()
#include <ctime>    // For clock()

// Sequential Merge Sort
void mergeSortSequential(int* arr, int left, int right) {
    if (left < right) {
        int mid = left + (right - left) / 2;
        mergeSortSequential(arr, left, mid);
        mergeSortSequential(arr, mid + 1, right);

        // Merge the sorted halves
        int temp[right - left + 1];
        int i = left, j = mid + 1, k = 0;

        while (i <= mid && j <= right) {
            if (arr[i] <= arr[j]) {
                temp[k++] = arr[i++];
            } else {
                temp[k++] = arr[j++];
            }
        }

        while (i <= mid) {
            temp[k++] = arr[i++];
        }

        while (j <= right) {
            temp[k++] = arr[j++];
        }

        for (int p = 0; p < k; p++) {
            arr[left + p] = temp[p];
        }
    }
}

// Parallel Merge Sort using CUDA
__global__ void merge(int* arr, int left, int mid, int right, int* temp) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    int i = left + tid;
    int j = mid + 1 + tid;
    int k = tid;

    while (i <= mid && j <= right) {
        if (arr[i] <= arr[j]) {
            temp[k] = arr[i];
            i += blockDim.x * gridDim.x;
        } else {
            temp[k] = arr[j];
            j += blockDim.x * gridDim.x;
        }
        k += blockDim.x * gridDim.x;
    }

    while (i <= mid) {
        temp[k] = arr[i];
        i += blockDim.x * gridDim.x;
        k += blockDim.x * gridDim.x;
    }

    while (j <= right) {
        temp[k] = arr[j];
        j += blockDim.x * gridDim.x;
        k += blockDim.x * gridDim.x;
    }

    __syncthreads();

    // Copy merged elements back to the original array
    for (int idx = tid; idx < k; idx += blockDim.x * gridDim.x) {
        arr[left + idx] = temp[idx];
    }
}

void mergeSortParallel(int* arr, int left, int right, int* temp) {
    if (left < right) {
        int mid = left + (right - left) / 2;
        mergeSortParallel(arr, left, mid, temp);
        mergeSortParallel(arr, mid + 1, right, temp);
        merge<<<1, 1>>>(arr, left, mid, right, temp);
    }
}

// Host function to invoke parallel merge sort
void parallelMergeSort(int* arr, int size) {
    int* d_arr;
    int* d_temp;
    cudaMalloc(&d_arr, size * sizeof(int));
    cudaMalloc(&d_temp, size * sizeof(int));
    cudaMemcpy(d_arr, arr, size * sizeof(int), cudaMemcpyHostToDevice);

    int left = 0;
    int right = size - 1;

    mergeSortParallel(d_arr, left, right, d_temp);
    cudaDeviceSynchronize();

    cudaMemcpy(arr, d_arr, size * sizeof(int), cudaMemcpyDeviceToHost);

    cudaFree(d_arr);
    cudaFree(d_temp);
}

int main() {
    const int size = 1000;
    int arr[size];
    for (int i = 0; i < size; i++) {
        arr[i] = rand() % 1000;  // Random numbers between 0 and 999
    }

    clock_t startSeqMerge = clock();
    mergeSortSequential(arr, 0, size - 1);
    clock_t endSeqMerge = clock();
    double timeSeqMerge = double(endSeqMerge - startSeqMerge) / CLOCKS_PER_SEC * 1000;

    clock_t startParMerge = clock();
    parallelMergeSort(arr, size);
    clock_t endParMerge = clock();
    double timeParMerge = double(endParMerge - startParMerge) / CLOCKS_PER_SEC * 1000;

    // Print execution times
    std::cout << "Parallel Merge Sort Time: " << timeSeqMerge << " milliseconds" << std::endl;
    std::cout << "Sequential Merge Sort Time: " << timeParMerge << " milliseconds" << std::endl;

    return 0;
}


Overwriting mer.cu


In [10]:
!nvcc mer.cu -o mer
!./mer

Parallel Merge Sort Time: 0.197 milliseconds
Sequential Merge Sort Time: 209.972 milliseconds
